# Vector Prediction

Vectorized Prediction algorithms for MERCS. We need that speed.

# Imports

In [1]:
from functools import partial
import numpy as np
import numba

from mercs.utils import DESC_ENCODING, TARG_ENCODING, MISS_ENCODING, code_to_query, get_att_2d

# Functions

## Helpers

In [2]:
# Strategies
def mi(m_codes, m_fimps, m_score, q_code, m_avl=None, random_state=997):

    # Init
    a_src, a_tgt, _ = code_to_query(q_code)

    # Criterion
    c_all = criterion(m_score, a_filter=a_tgt, m_filter=m_avl, aggregation=None)

    # Pick
    m_sel = pick(c_all, thresholds=None)

    return m_sel


def mrai(
    m_codes,
    m_fimps,
    m_score,
    q_code=None,
    a_src=None,
    a_tgt=None,
    m_avl=None,
    init_threshold=1.0,
    stepsize=0.1,
    any_target=False,
    stochastic=False,
    thresholds=None,
    random_state=997,
):
    # Init
    if m_avl is None:
        m_avl = np.arange(m_codes.shape[0], dtype=np.int16)

    if thresholds is None:
        thresholds = _init_thresholds(init_threshold, stepsize)

    if a_src is None or a_tgt is None:
        a_src, a_tgt, _ = code_to_query(q_code, return_sets=True)

    # Criterion
    c_src = criterion(m_fimps, a_filter=a_src, m_filter=m_avl, aggregation=True)
    c_tgt = criterion(m_score, a_filter=a_tgt, m_filter=m_avl, aggregation=None)
    c_all = c_src.reshape(-1, 1) * c_tgt + c_tgt

    # Pick
    m_sel = pick(
        c_all,
        thresholds=thresholds,
        any_target=any_target,
        stochastic=stochastic,
        random_state=random_state,
    )

    return m_sel


def it(
    m_codes,
    m_fimps,
    m_score,
    q_code,
    m_avl=None,
    max_steps=4,
    init_threshold=1.0,
    stepsize=0.1,
    random_state=997,
):
    m_sel = []
    thresholds = _init_thresholds(init_threshold, stepsize)
    any_target = True
    stochastic = False

    q_desc, q_targ, q_miss = code_to_query(q_code)
    a_src = q_desc
    a_tgt = np.hstack([q_targ, q_miss])

    if m_avl is None:
        m_avl = np.arange(m_codes.shape[0], dtype=np.int16)

    for step in range(max_steps):

        # Check if this is our last chance
        last = step + 1 == max_steps
        if last:
            any_target = False  # Finish the job
            a_tgt = np.setdiff1d(
                q_targ, a_src
            )  # Focus exclusively on non-predicted q_targ attributes.

        step_m_sel = mrai(
            m_codes,
            m_fimps,
            m_score,
            a_src=a_src,
            a_tgt=a_tgt,
            m_avl=m_avl,
            stochastic=stochastic,
            any_target=any_target,
            thresholds=thresholds,
            random_state=random_state,
        )

        a_prd = get_att_2d(m_codes[step_m_sel, :], kind="targ")
        
        a_src = np.union1d(a_src, a_prd)
        a_tgt = np.setdiff1d(a_tgt, a_prd)

        m_avl = np.setdiff1d(m_avl, step_m_sel)
        m_sel.append(step_m_sel)

        if _stopping_criterion_it(q_targ, a_src):
            break

        if len(step_m_sel) == 0:
            raise ValueError(
                "No progress was made. This indicates an impossible query."
            )

    return m_sel


# MRAI-IT-RW Criterion
def criterion(m_matrix, m_filter, a_filter, aggregation=None):
    """
    Typical usecase 
    
    m_matrix = m_fimps
    
    m_filter = available models
    a_filter = available attributes.
    
    """
    nb_rows = m_matrix.shape[0]
    nb_cols = len(a_filter)
    
    c_matrix = np.zeros((nb_rows, nb_cols), dtype=np.float32)

    m_idx = a_filter + m_filter.reshape(-1, 1) * m_matrix.shape[1]
    c_matrix[m_filter, :] = m_matrix.take(m_idx.flat).reshape(nb_rows, nb_cols)

    if aggregation is None:
        return c_matrix
    else:
        return np.sum(c_matrix, axis=1).reshape(-1, 1).astype(np.float32)


# Picks
def pick(
    criteria, thresholds=None, any_target=False, stochastic=False, random_state=997
):

    if thresholds is None:
        return np.where(criteria >= 0)[0]
    else:
        m_sel = []

        picking_function = _stochastic_pick if stochastic else _greedy_pick

        if any_target:
            criteria = np.max(criteria, axis=1).reshape(-1, 1)

        for c_idx in range(criteria.shape[1]):
            m_sel.extend(
                picking_function(
                    criteria[:, c_idx], thresholds=thresholds, random_state=random_state
                )
            )

        return np.unique(m_sel)


def _greedy_pick(c_all, thresholds=None, **kwargs):
    for thr in thresholds:
        m_sel = np.where(c_all > thr)[0]
        if _stopping_criterion_greedy_pick(m_sel):
            break
    return m_sel


def _stochastic_pick(c_all, random_state=997, **kwargs):
    np.random.seed(random_state)
    norm = np.linalg.norm(c_all, 1)

    if norm > 0:
        distribution = c_all / np.sum(c_all)
    else:
        distribution = np.full(len(c_all), 1 / len(c_all))

    draw = np.random.multinomial(1, distribution, size=1)
    return np.where(draw == 1)[1]


# Stopping Criteria
def _stopping_criterion_it(q_targ, a_src):
    return np.setdiff1d(q_targ, a_src).shape[0] == 0


def _stopping_criterion_greedy_pick(m_sel):
    return len(m_sel) > 0


In [3]:
# Helpers
def _init_thresholds(init_threshold, stepsize):
    return np.arange(init_threshold, -stepsize, -stepsize, dtype=np.float32)

# Sandbox



## Setup

In [4]:
def init_m_fimps_or_m_scores(m_codes, kind='m_fimps'):
    
    if kind in {'m_fimps'}:
        value = DESC_ENCODING
    elif kind in {'m_score'}:
        value = TARG_ENCODING
    
    m_init = np.zeros(m_codes.shape)
    m_init[np.where(m_codes == value)] = np.random.rand(len(m_init[np.where(m_codes == value)]))

    normalize(m_init, norm='l1', copy=False)
    
    #m_init = m_init*100
    return m_init.astype(np.float32)

In [5]:
from sklearn.preprocessing import normalize

nb_attributes = 2*10**3
nb_iterations = 2

m_codes = np.random.randint(-1,2, size=(nb_iterations*nb_attributes, nb_attributes), dtype=np.int8)
q_code = np.random.randint(-1,2, nb_attributes, dtype=np.int8)


m_fimps = init_m_fimps_or_m_scores(m_codes, kind='m_fimps')
m_score = init_m_fimps_or_m_scores(m_codes, kind='m_score')

#m_fimps, m_score

msg = """
m_codes:
{}
m_fimps:
{}
m_score:
{}
q_code:
{}
""".format(m_codes, m_fimps, m_score, q_code)
print(msg)


m_codes:
[[ 1  1  1 ...  0 -1  0]
 [-1 -1  0 ... -1  1  0]
 [ 0  1  0 ...  1  0 -1]
 ...
 [ 0  1  1 ...  1  0  1]
 [-1  0  0 ...  0  0  0]
 [ 1  0 -1 ...  1  0  0]]
m_fimps:
[[0.         0.         0.         ... 0.00151566 0.         0.00050955]
 [0.         0.         0.00284598 ... 0.         0.         0.000933  ]
 [0.00039445 0.         0.00154299 ... 0.         0.0022003  0.        ]
 ...
 [0.00123323 0.         0.         ... 0.         0.00274897 0.        ]
 [0.         0.00273181 0.00080517 ... 0.00253666 0.00232512 0.00039437]
 [0.         0.00022585 0.         ... 0.         0.00032463 0.00029425]]
m_score:
[[3.5412965e-04 1.5658023e-03 1.9886063e-03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  3.1607199e-04 0.0000000e+00]
 [0.0000000e+00 8.2027511e-04 0.0000000e+00 ... 2.4865987e-03
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 1.0674543e-03 1.3851456e-03 ... 9.9036051e-04
  0.0000000e+00 6.8148423e

## Tests

In [6]:
6.33

6.33

In [7]:
%%timeit
it(m_codes, m_fimps, m_score, q_code, m_avl=None)

174 ms ± 1.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
it(m_codes, m_fimps, m_score, q_code, m_avl=None)

172 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


NameError: name 'a1_rows' is not defined

In [12]:
a1 = np.array([[1,2,3],[4,5,6],[7,8,9]])
a2 = np.array([[4,5,6],[7,8,9],[1,1,1]])
a1_rows = a1.view([('', a1.dtype)] * a1.shape[1])
a2_rows = a2.view([('', a2.dtype)] * a2.shape[1])

In [13]:
%%timeit
np.setdiff1d(a1_rows, a2_rows).view(a1.dtype).reshape(-1, a1.shape[1])


47.8 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
%%timeit
np.setdiff1d(a1, a2)

30.9 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
